In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库
import time

In [15]:
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

In [16]:
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']
    
    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)
    
    return data

In [17]:
data = pd.read_csv('train.csv',index_col=False)
data2 = pd.read_csv('test.csv',index_col=False) 
data3 = pd.read_csv('test1.csv',index_col=False) 

In [18]:
data = pre_process(data)
data2 = pre_process(data2)
data3 = pre_process(data3)

item_category_list_ing
item_property_list_ing
context_timestamp_ing
item_category_list_ing
item_property_list_ing
context_timestamp_ing
item_category_list_ing
item_property_list_ing
context_timestamp_ing


In [19]:
no  = data.pop('is_trade')

In [20]:
data

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2
0,108641074714126964,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,1,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
1,5754713551599725161,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,0,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
2,842679481291040981,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,0,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
3,937088850059189027,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,2681414445369714628,1,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
4,7975697065017708072,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,2729475788342039013,0,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
5,7764762765372067286,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,4512655448325954611,1,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
6,6956333474094867789,285660928590172217,9057103201734987852,548352491538518780,8,9,8,13,8811056487516803043,1,...,5012,0.974878,0.976863,0.969278,7908382889764677758,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413
7,8387099821892927911,285660928590172217,9057103201734987852,548352491538518780,8,9,8,13,6507704883896466138,0,...,5012,0.974878,0.976863,0.969278,7908382889764677758,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413
8,4021878205550012615,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,16,6203308008480593423,0,...,5012,0.974878,0.976863,0.969278,7908382889764677758,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413
9,6499571365974135517,285660928590172217,9057103201734987852,548352491538518780,8,9,8,13,6041712044514783312,0,...,5012,0.974878,0.976863,0.969278,7908382889764677758,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413


In [21]:
data_all = data.append(data2)
data_all = data_all.append(data3)
print(data.shape)
print(data2.shape)
print(data_all.shape)

(478087, 30)
(18371, 30)
(539346, 30)


In [22]:
data_all = data_all.reset_index()
data_all.pop('index')
data_all

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2
0,108641074714126964,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,1,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
1,5754713551599725161,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,0,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
2,842679481291040981,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,0,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
3,937088850059189027,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,2681414445369714628,1,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
4,7975697065017708072,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,2729475788342039013,0,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
5,7764762765372067286,3412720377098676069,1975590437749032870,3948283326616421003,3,3,4,14,4512655448325954611,1,...,5002,1.000000,1.000000,1.000000,7908382889764677758,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413
6,6956333474094867789,285660928590172217,9057103201734987852,548352491538518780,8,9,8,13,8811056487516803043,1,...,5012,0.974878,0.976863,0.969278,7908382889764677758,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413
7,8387099821892927911,285660928590172217,9057103201734987852,548352491538518780,8,9,8,13,6507704883896466138,0,...,5012,0.974878,0.976863,0.969278,7908382889764677758,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413
8,4021878205550012615,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,16,6203308008480593423,0,...,5012,0.974878,0.976863,0.969278,7908382889764677758,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413
9,6499571365974135517,285660928590172217,9057103201734987852,548352491538518780,8,9,8,13,6041712044514783312,0,...,5012,0.974878,0.976863,0.969278,7908382889764677758,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413


In [23]:
data_all.columns

Index(['instance_id', 'item_id', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'category_0', 'category_1', 'category_2',
       'property_0', 'property_1', 'property_2'],
      dtype='object')

In [24]:
data_all = data_all.sort_values('context_timestamp')

In [25]:
data_all[['context_timestamp']]

,context_timestamp
3844,2018-09-18 00:00:01
24042,2018-09-18 00:00:04
59325,2018-09-18 00:00:05
70760,2018-09-18 00:00:11
27192,2018-09-18 00:00:14
76643,2018-09-18 00:00:14
59192,2018-09-18 00:00:16
57454,2018-09-18 00:00:16
24475,2018-09-18 00:00:17
23297,2018-09-18 00:00:17


In [26]:
data_all[['user_id','category_1','instance_id']].groupby(['user_id','category_1']).count()

instance_id
user_id             category_1                      
7209196426168       8277336076276184272            1
24779788309075      7258015885215914736            3
36134987234568      5799347067982556520            1
59341486148291      8277336076276184272            1
174676690398750     8277336076276184272            1
179317972644611     8277336076276184272            8
179451404933115     5799347067982556520            1
                    8277336076276184272            7
193133497894650     8277336076276184272            1
212049717332865     8277336076276184272            1
217191929432135     509660095530134768             1
                    5755694407684602296            3
                    8277336076276184272            2
218627666998650     509660095530134768             1
                    8277336076276184272            1
225768902336823     8277336076276184272            1
300164043453670     5799347067982556520            1
317896740941222     7258015885215914736            1
343687024536467     509660095530134768             1
                    7258015885215914736            1
343846706165657     8277336076276184272            1
397285691292122     5755694407684602296            2
471191193416668     509660095530134768             4
506604787274365     8277336076276184272            2
510416141439025     8277336076276184272            1
523117691035117     8277336076276184272            1
547560177346870     8277336076276184272            1
601357023584293     8277336076276184272            1
635591985978049     5799347067982556520            1
636099518731252     8277336076276184272            1
...                                              ...
9222418414773457583 8277336076276184272            1
9222476565403393762 8277336076276184272            6
9222500199833617909 7258015885215914736            2
9222513822757222000 8277336076276184272            1
9222553786417581634 5755694407684602296            2
                    5799347067982556520            7
                    8277336076276184272            2
9222567600393382449 509660095530134768             2
9222660336380554559 7258015885215914736            2
9222661010396903891 7258015885215914736            2
9222740782320901996 7258015885215914736            1
9222741368620686467 7258015885215914736            1
9222790095407479120 8277336076276184272            1
9222795826502953393 5755694407684602296            1
9222798158450049253 509660095530134768             1
                    5799347067982556520            5
9222878895135397908 8277336076276184272            1
9223077942117037524 5799347067982556520            1
9223099985366849305 5755694407684602296            1
9223214422415101680 5799347067982556520            1
                    8277336076276184272            8
9223262087638217435 509660095530134768             1
                    5755694407684602296            6
9223263001143284782 509660095530134768             1
9223286667125380235 2011981573061447208            1
9223296966028830603 3203673979138763595            3
9223324018512912014 7258015885215914736            1
9223335008361681580 5755694407684602296            2
                    8277336076276184272            3
9223345267743287634 8277336076276184272            1

[294224 rows x 1 columns]

In [27]:
data_all['category_1_pvnum'] = 0
for name,num in data_all.groupby(['user_id','category_1']):
#     print(name)
#     print(num)
#     print()
# #     print(len(slist))
# #     print(slist.iloc[0,-1])
# #     slist.iloc[0,-1] = 1
# #     i=-2
#     print(num[['context_timestamp']])
    num = num.reset_index().set_index(['index']).sort_values('context_timestamp')
#     print()
#     print(num[['context_timestamp']])
    for i in range(len(num)):
#         data1
#         print(num.iloc[i,0])
#         print(num.index[i])
        data_all.loc[num.index[i],'category_1_pvnum'] = (i+1)
#         print(data_all.loc[num.index[i],'category_1_pvnum'])
#         slist.iloc[i,2] = 1/100*(i+1)
#         print(slist.iloc[i,2])
#         print(1/100*(i+1))
#         i = i + 1
#         print(s)
#     print()
#     break

In [28]:
userclickdata = pd.merge(data[['instance_id']],data_all[['category_1_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata2 = pd.merge(data2[['instance_id']],data_all[['category_1_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata3 = pd.merge(data3[['instance_id']],data_all[['category_1_pvnum','instance_id']],on=['instance_id'],how='left')

In [29]:
userclickdata.shape

(478087, 2)

In [30]:
data.shape

(478087, 30)

In [31]:
userclickdata

,instance_id,category_1_pvnum
0,108641074714126964,1
1,5754713551599725161,3
2,842679481291040981,1
3,937088850059189027,1
4,7975697065017708072,1
5,7764762765372067286,2
6,6956333474094867789,11
7,8387099821892927911,1
8,4021878205550012615,1
9,6499571365974135517,4


In [32]:
print(userclickdata.shape)
print(userclickdata2.shape)
print(userclickdata3.shape)

(478087, 2)
(18371, 2)
(42888, 2)


In [33]:
userclickdata[['instance_id','category_1_pvnum']].to_csv('train_user_clickpv.csv',index=False) 
userclickdata2[['instance_id','category_1_pvnum']].to_csv('test_user_clickpv.csv',index=False) 
userclickdata3[['instance_id','category_1_pvnum']].to_csv('test1_user_clickpv.csv',index=False) 

In [34]:
data_all['property_0_pvnum'] = 0
for name,num in data_all.groupby(['user_id','property_0']):
#     print(name)
#     print(num)
#     print()
# #     print(len(slist))
# #     print(slist.iloc[0,-1])
# #     slist.iloc[0,-1] = 1
# #     i=-2
#     print(num[['context_timestamp']])
    num = num.reset_index().set_index(['index']).sort_values('property_0_pvnum')
#     print()
#     print(num[['context_timestamp']])
    for i in range(len(num)):
#         data1
#         print(num.iloc[i,0])
#         print(num.index[i])
        data_all.loc[num.index[i],'property_0_pvnum'] = (i+1)
#         print(data_all.loc[num.index[i],'category_1_pvnum'])
#         slist.iloc[i,2] = 1/100*(i+1)
#         print(slist.iloc[i,2])
#         print(1/100*(i+1))
#         i = i + 1
#         print(s)
#     print()
#     break

In [35]:
userclickdata = pd.merge(data[['instance_id']],data_all[['property_0_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata2 = pd.merge(data2[['instance_id']],data_all[['property_0_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata3 = pd.merge(data3[['instance_id']],data_all[['property_0_pvnum','instance_id']],on=['instance_id'],how='left')

In [36]:
print(userclickdata.shape)
print(userclickdata2.shape)
print(userclickdata3.shape)

(478087, 2)
(18371, 2)
(42888, 2)


In [37]:
userclickdata[['instance_id','property_0_pvnum']].to_csv('train_property_0_pvnum.csv',index=False) 
userclickdata2[['instance_id','property_0_pvnum']].to_csv('test_property_0_pvnum.csv',index=False) 
userclickdata3[['instance_id','property_0_pvnum']].to_csv('test1_property_0_pvnum.csv',index=False) 

In [38]:
data_all['property_1_pvnum'] = 0
for name,num in data_all.groupby(['user_id','property_1']):
#     print(name)
#     print(num)
#     print()
# #     print(len(slist))
# #     print(slist.iloc[0,-1])
# #     slist.iloc[0,-1] = 1
# #     i=-2
#     print(num[['context_timestamp']])
    num = num.reset_index().set_index(['index']).sort_values('context_timestamp')
#     print()
#     print(num[['context_timestamp']])
    for i in range(len(num)):
#         data1
#         print(num.iloc[i,0])
#         print(num.index[i])
        data_all.loc[num.index[i],'property_1_pvnum'] = (i+1)
#         print(data_all.loc[num.index[i],'category_1_pvnum'])
#         slist.iloc[i,2] = 1/100*(i+1)
#         print(slist.iloc[i,2])
#         print(1/100*(i+1))
#         i = i + 1
#         print(s)
#     print()
#     break

In [39]:
userclickdata = pd.merge(data[['instance_id']],data_all[['property_1_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata2 = pd.merge(data2[['instance_id']],data_all[['property_1_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata3 = pd.merge(data3[['instance_id']],data_all[['property_1_pvnum','instance_id']],on=['instance_id'],how='left')

In [40]:
print(userclickdata.shape)
print(userclickdata2.shape)
print(userclickdata3.shape)

(478087, 2)
(18371, 2)
(42888, 2)


In [41]:
userclickdata[['instance_id','property_1_pvnum']].to_csv('train_property_1_pvnum.csv',index=False) 
userclickdata2[['instance_id','property_1_pvnum']].to_csv('test_property_1_pvnum.csv',index=False) 
userclickdata3[['instance_id','property_1_pvnum']].to_csv('test1_property_1_pvnum.csv',index=False) 

In [42]:
data_all['property_2_pvnum'] = 0
for name,num in data_all.groupby(['user_id','property_2']):
#     print(name)
#     print(num)
#     print()
# #     print(len(slist))
# #     print(slist.iloc[0,-1])
# #     slist.iloc[0,-1] = 1
# #     i=-2
#     print(num[['context_timestamp']])
    num = num.reset_index().set_index(['index']).sort_values('context_timestamp')
#     print()
#     print(num[['context_timestamp']])
    for i in range(len(num)):
#         data1
#         print(num.iloc[i,0])
#         print(num.index[i])
        data_all.loc[num.index[i],'property_2_pvnum'] = (i+1)
#         print(data_all.loc[num.index[i],'category_1_pvnum'])
#         slist.iloc[i,2] = 1/100*(i+1)
#         print(slist.iloc[i,2])
#         print(1/100*(i+1))
#         i = i + 1
#         print(s)
#     print()
#     break

In [43]:
userclickdata = pd.merge(data[['instance_id']],data_all[['property_2_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata2 = pd.merge(data2[['instance_id']],data_all[['property_2_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata3 = pd.merge(data3[['instance_id']],data_all[['property_2_pvnum','instance_id']],on=['instance_id'],how='left')

In [44]:
print(userclickdata.shape)
print(userclickdata2.shape)
print(userclickdata3.shape)

(478087, 2)
(18371, 2)
(42888, 2)


In [45]:
userclickdata[['instance_id','property_2_pvnum']].to_csv('train_property_2_pvnum.csv',index=False) 
userclickdata2[['instance_id','property_2_pvnum']].to_csv('test_property_2_pvnum.csv',index=False) 
userclickdata3[['instance_id','property_2_pvnum']].to_csv('test1_property_2_pvnum.csv',index=False) 

In [46]:
data_all['item_id_pvnum'] = 0
for name,num in data_all.groupby(['user_id','item_id']):
#     print(name)
#     print(num)
#     print()
# #     print(len(slist))
# #     print(slist.iloc[0,-1])
# #     slist.iloc[0,-1] = 1
# #     i=-2
#     print(num[['context_timestamp']])
    num = num.reset_index().set_index(['index']).sort_values('context_timestamp')
#     print()
#     print(num[['context_timestamp']])
    for i in range(len(num)):
#         data1
#         print(num.iloc[i,0])
#         print(num.index[i])
        data_all.loc[num.index[i],'item_id_pvnum'] = (i+1)
#         print(data_all.loc[num.index[i],'category_1_pvnum'])
#         slist.iloc[i,2] = 1/100*(i+1)
#         print(slist.iloc[i,2])
#         print(1/100*(i+1))
#         i = i + 1
#         print(s)
#     print()
#     break

In [47]:
userclickdata = pd.merge(data[['instance_id']],data_all[['item_id_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata2 = pd.merge(data2[['instance_id']],data_all[['item_id_pvnum','instance_id']],on=['instance_id'],how='left')
userclickdata3 = pd.merge(data3[['instance_id']],data_all[['item_id_pvnum','instance_id']],on=['instance_id'],how='left')

In [48]:
print(userclickdata.shape)
print(userclickdata2.shape)
print(userclickdata3.shape)

(478087, 2)
(18371, 2)
(42888, 2)


In [49]:
userclickdata[['instance_id','item_id_pvnum']].to_csv('train_item_id_pvnum.csv',index=False) 
userclickdata2[['instance_id','item_id_pvnum']].to_csv('test_item_id_pvnum.csv',index=False) 
userclickdata3[['instance_id','item_id_pvnum']].to_csv('test1_item_id_pvnum.csv',index=False)